In [2]:
import json
from pathlib import Path
corpus_name = 'conll2003'
model_name = 'bert'
variant = f'{corpus_name}_{model_name}'

# corpus_file = '/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/ExperimentData/corpora.json'
corpus_file = '/Users/ahmed/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/ExperimentData/corpora.json'


with open(corpus_file, 'r') as file:
    corpora = json.load(file)  # Use json.load() to read file, not json.loads()




In [3]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px
from seqeval.metrics.sequence_labeling import get_entities
from seqeval.scheme import Entities, auto_detect

from collections import Counter, defaultdict

from collections import Counter
import pandas as pd


In [231]:
# managers/tabs/dataset_helpers.py

class EntityTagOOVRate(BaseDatasetHelper):
    """
    Per-tag OOV, computed train→test for each dataset.
    Output rows: Dataset, Tag, OOV Words Count, Total Unique Words in Test, OOV Rate
    """

    def __init__(self, corpora: Dict, tags: Optional[List[str]] = None):
        super().__init__(corpora)
        self.tags = tags or _DEFAULT_TAGS

    def _unique_words_per_tag(self, split_sentences, tags: List[str]) -> Dict[str, set]:
        buckets = {t: set() for t in tags}
        for s in split_sentences:
            for w, t in zip(s["words"], s["tags"]):
                t = self._normalize_tag(t)  # e.g., PERS→PER
                if t in buckets:
                    buckets[t].add(w)
        return buckets

    def generate_df(self, selected_variant: str) -> pd.DataFrame:
        rows: List[dict] = []
        seen_labels = set()

        for ds_key in self.resolve_dataset_keys(selected_variant):
            ds_lbl = self.ds_label(ds_key)
            if ds_lbl in seen_labels:
                continue
            seen_labels.add(ds_lbl)

            splits = self.corpora[ds_key]["splits"]
            train = splits.get("train")
            test  = splits.get("test")
            if not train or not test:
                # skip silently; or emit zeros if you prefer
                continue

            train_buckets = self._unique_words_per_tag(train, self.tags)
            test_buckets  = self._unique_words_per_tag(test,  self.tags)

            for tag in self.tags:
                test_set  = test_buckets.get(tag, set())
                train_set = train_buckets.get(tag, set())
                oov_set   = test_set - train_set
                oov_rate  = (len(oov_set) / len(test_set)) if test_set else 0.0

                rows.append({
                    "Dataset": ds_lbl,
                    "Tag": tag,
                    "OOV Words Count": float(len(oov_set)),
                    "Total Unique Words in Test": float(len(test_set)),
                    "OOV Rate": round(oov_rate, 4),
                })

        return pd.DataFrame(rows, columns=[
            "Dataset", "Tag", "OOV Words Count", "Total Unique Words in Test", "OOV Rate"
        ])


In [232]:
# managers/plotting/dataset_plotting_managers.py
import plotly.express as px
from dash import html, dcc

def plot_entity_tag_oov_bar(
    df: pd.DataFrame,
    *,
    title: str = "Per-Tag OOV Rate (Train→Test)",
    tag_order: list[str] | None = None,
    show_values: bool = True,
) -> html.Div:
    data = df.copy()

    # keep a stable tag order if provided
    if tag_order and "Tag" in data.columns:
        cats = [t for t in tag_order if t in data["Tag"].unique().tolist()]
        if cats:
            data["Tag"] = pd.Categorical(data["Tag"], categories=cats, ordered=True)

    # detect proportion (0..1) to format as %
    looks_like_prop = (
        "OOV Rate" in data.columns
        and pd.api.types.is_numeric_dtype(data["OOV Rate"])
        and data["OOV Rate"].dropna().between(0.0, 1.0).all()
    )

    fig = px.bar(
        data,
        x="Tag",
        y="OOV Rate",
        color="Dataset",
        barmode="group",
        text="OOV Rate" if show_values else None,
        category_orders={"Tag": list(data["Tag"].cat.categories) if hasattr(data["Tag"], "cat") else None},
        labels={"Tag": "Entity Tag", "OOV Rate": "OOV Rate"},
        title=title,
    )

    if show_values:
        if looks_like_prop:
            fig.update_traces(texttemplate="%{y:.2%}", textposition="outside", cliponaxis=False)
        else:
            fig.update_traces(texttemplate="%{y:.4f}", textposition="outside", cliponaxis=False)

    if looks_like_prop:
        fig.update_yaxes(tickformat=".0%", rangemode="tozero")
    else:
        fig.update_yaxes(rangemode="tozero")

    fig.update_layout(
        template="plotly_white",
        margin=dict(t=60, l=20, r=20, b=40),
        legend_title=None,
        bargap=0.25,
        height=600,
    )
    fig.update_xaxes(tickangle=-30)

    return fig


In [234]:
helper = EntityTagOOVRate(corpora=corpora)
df = helper.generate_df("combined")
df
plot_entity_tag_oov_bar(
            df,
            title="Entity Tag OOV Rate (Train→Test) Across Datasets",
            tag_order=_DEFAULT_TAGS,
            show_values=True,
        )

In [134]:
combined_general_stats, combined_tag_stats = calculate_combined_stats(corpora)


In [195]:
combined_general_stats

,Split,Total Tokens,Unique Words,TTR,Entity Words,NE Proportion,Entity Types,ETTR,Dataset
0,train,125102,29252,0.2338,13181,0.1054,4069,0.3087,ANERCorp_CamelLab
1,test,25008,9075,0.3629,3375,0.1350,1603,0.4750,ANERCorp_CamelLab
2,Total,150110,32714,0.2179,16556,0.1103,5026,0.3036,ANERCorp_CamelLab
3,train,203621,23623,0.1160,34043,0.1672,8287,0.2434,CoNLL-2003
4,test,46435,9488,0.2043,8112,0.1747,3129,0.3857,CoNLL-2003
5,Total,250056,27316,0.1092,42155,0.1686,9819,0.2329,CoNLL-2003


In [170]:
combined_general_stats

,Split,Total Tokens,Unique Words,TTR,Entity Words,NE Proportion,Entity Types,ETTR,Dataset
0,train,125102,29252,0.2338,13181,0.1054,4069,0.3087,ANERCorp_CamelLab
1,test,25008,9075,0.3629,3375,0.1350,1603,0.4750,ANERCorp_CamelLab
2,Total,150110,32714,0.2179,16556,0.1103,5026,0.3036,ANERCorp_CamelLab
3,train,203621,23623,0.1160,34043,0.1672,8287,0.2434,CoNLL-2003
4,test,46435,9488,0.2043,8112,0.1747,3129,0.3857,CoNLL-2003
5,Total,250056,27316,0.1092,42155,0.1686,9819,0.2329,CoNLL-2003


In [8]:
oov_stats = calculate_oov_rates(corpora)
pd.DataFrame(oov_stats)


,ANERCorp_CamelLab,conll2003
OOV Words Count,3462.0000,3693.0000
Total Unique Words in Test,9075.0000,9488.0000
OOV Rate,0.3815,0.3892


# Token Level

In [93]:

# Example usage
# df is your DataFrame loaded with the necessary 'Dataset', 'Tag', 'Words Proportion', and 'Tag Words' columns
visualizer = EntityTagVisualizer(df=combined_tag_stats)
visualizer.preprocess_data()
visualizer.plot_faceted_bar_chart('Words Proportion', 'Tag Words', "Entity Tag Words Distribution Across Training and Testing Splits",)


In [184]:
# Step 1: Calculate the total tag types for each Dataset and Split
total_tag_types = combined_tag_stats.groupby(['Dataset', 'Split'])['Tag Types'].sum().reset_index()
total_tag_types.rename(columns={'Tag Types': 'Total Tag Types'}, inplace=True)

# Step 2: Merge the total back to the original DataFrame to calculate proportions
df = combined_tag_stats.merge(total_tag_types, on=['Dataset', 'Split'])

# Step 3: Calculate the proportion of tag types within each group
df['Tag Type Proportion'] = df['Tag Types'] / df['Total Tag Types']

df
visualizer = EntityTagVisualizer(df=df)
visualizer.preprocess_data()

visualizer.plot_faceted_bar_chart('Tag Type Proportion', 'Tag Types', "Entity Tag Word Types Distribution Across Training and Testing Splits",)

In [5]:

# Usage example
tags = ['B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-MISC', 'I-MISC', 'O']

overlap_matrix = OverlapMatrixOneScale(corpora, tags)
# overlap_matrix.visualize_matrices({'ANERCorp': 'ANERCorp_CamelLab', 'CoNLL-2003': 'conll2003'}, 'train')
overlap_matrix.visualize_matrices_one_scale({'ANERCorp': 'ANERCorp_CamelLab', 'CoNLL-2003': 'conll2003'}, 'train')


overlap_matrix = OverlapMatrixOneScale(corpora, tags)
# overlap_matrix.visualize_matrices({'ANERCorp': 'ANERCorp_CamelLab', 'CoNLL-2003': 'conll2003'}, 'test')
overlap_matrix.visualize_matrices_one_scale({'ANERCorp': 'ANERCorp_CamelLab', 'CoNLL-2003': 'conll2003'}, 'test')



In [8]:
visualizer = EntityTagTTRVisualizer(df)
visualizer.preprocess_data()
visualizer.plot_faceted_bar_chart(
    metric='TWR',
    text='TWR',
    title='Type-to-Word Ratio (TWR) Across Entity Tags in ANERCorp and CoNLL-2003 (Train and Test Splits)'
)

In [9]:

# Example Usage
# Assuming `corpora` is already loaded
datasets_of_interest = ['ANERCorp_CamelLab', 'conll2003']
tags_of_interest = ['B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-MISC', 'I-MISC']


analyzer = EntityTagDistributionAnalyzer(corpora, datasets_of_interest, tags_of_interest)

# Calculate distributions for both training and testing splits
distributions = analyzer.calculate_distributions(['train', 'test'])

# Visualize the standard deviation across splits
analyzer.visualize_distributions(
    distributions,
    metric='Standard Deviation',
    title="Standard Deviation of Word Type Frequencies Across Entity Tags (Train and Test Splits)",
)


In [10]:
pd.DataFrame(Counter([w for sen in corpora['ANERCorp_CamelLab']['splits']['train'] for w,t in zip(sen['words'], sen['tags']) if t =='I-ORG']).items()).sort_values(by=1, ascending=False).head(50)

,0,1
7,المتحدة,178
6,الله,93
2,الأمن,49
29,العربية,21
57,الدولي,20
47,الامن,19
96,يونايتد,18
400,التجاري,17
3,الدولية,15
28,الدول,13


# OOV

In [11]:
# Assuming `corpora` is already loaded
datasets_of_interest = ['ANERCorp_CamelLab', 'conll2003']
tags_of_interest = ['B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-MISC', 'I-MISC']
dataset_mapping = {
    'ANERCorp_CamelLab': 'ANERCorp',
    'conll2003': 'CoNLL-2003'
    }

# Initialize the OOV Analyzer
oov_analyzer = OOVAnalyzer(corpora, datasets_of_interest, tags_of_interest)

# Compute OOV Rates
oov_df = oov_analyzer.calculate_oov_rates()

# Visualize OOV Rates
oov_analyzer.visualize_oov_rates(oov_df, dataset_mapping=dataset_mapping)


In [12]:
oov_stats_per_tag = calculate_oov_rates_per_tag(corpora)

# Convert results to a suitable format for display, such as a DataFrame
import pandas as pd

# Transform the dictionary into a DataFrame for easier viewing and analysis
data_frames = {}
for corpus, tags_data in oov_stats_per_tag.items():
    if isinstance(tags_data, dict):  # Ensure there's data to process
        df = pd.DataFrame.from_dict(tags_data, orient='index')
        data_frames[corpus] = df

# Creating a single DataFrame from the dictionary
all_data = pd.concat({
    k: v for k, v in data_frames.items()
}, axis=0)

# Rename the levels for clarity
all_data.index.names = ['Corpus', 'Tag']
all_data

OOV Words Count  Total Unique Words in Test  \
Corpus            Tag                                                   
ANERCorp_CamelLab B-LOC                98                         268   
                  B-PER               347                         506   
                  I-PER               288                         419   
                  B-ORG               131                         210   
                  I-LOC                18                          34   
                  I-ORG               106                         151   
                  B-MISC               93                         123   
                  I-MISC               93                         111   
conll2003         B-LOC               187                         475   
                  B-PER               519                         857   
                  I-PER               610                         797   
                  I-LOC                47                         102   
                  B-MISC              150                         274   
                  I-MISC               50                          94   
                  B-ORG               267                         658   
                  I-ORG               147                         379   

                          OOV Rate  
Corpus            Tag               
ANERCorp_CamelLab B-LOC     0.3657  
                  B-PER     0.6858  
                  I-PER     0.6874  
                  B-ORG     0.6238  
                  I-LOC     0.5294  
                  I-ORG     0.7020  
                  B-MISC    0.7561  
                  I-MISC    0.8378  
conll2003         B-LOC     0.3937  
                  B-PER     0.6056  
                  I-PER     0.7654  
                  I-LOC     0.4608  
                  B-MISC    0.5474  
                  I-MISC    0.5319  
                  B-ORG     0.4058  
                  I-ORG     0.3879

In [13]:
dataset = 'conll2003'
entity_tag = 'B-PER'
train = set([w for sen in corpora[dataset]['splits']['train'] for w, t in zip(sen['words'], sen['tags']) if t == entity_tag])
test = set([w for sen in corpora[dataset]['splits']['test'] for w, t in zip(sen['words'], sen['tags']) if t == entity_tag])
len(test - train)

519

# Entity

In [14]:
# Define parameters
datasets_of_interest = ['ANERCorp_CamelLab', 'conll2003']
schemes = ['IOB1', 'IOB2']
splits = ['train', 'test']
dataset_mapping = {
    'ANERCorp_CamelLab': 'ANERCorp',
    'conll2003': 'CoNLL-2003'
    }


# Initialize Analyzer
analyzer = EntityDistributionAnalyzer(corpora, datasets_of_interest, schemes, splits)

# Compute entity distributions
entity_distribution_df = analyzer.get_entity_distribution()

# Visualize entity distributions
analyzer.visualize_entity_distribution(entity_distribution_df, 'Proportion', dataset_mapping)


In [15]:
# Define parameters
datasets_of_interest = ['ANERCorp_CamelLab', 'conll2003']
schemes = ['IOB1', 'IOB2']
splits = ['train', 'test']
dataset_mapping = {
    'ANERCorp_CamelLab': 'ANERCorp',
    'conll2003': 'CoNLL-2003'
    }

# Initialize Analyzer
length_analyzer = EntityLengthAnalyzer(corpora, datasets_of_interest, schemes, splits)

# Compute entity length distributions
entity_length_df = length_analyzer.process_datasets()

# Visualize entity length distributions
length_analyzer.visualize_entity_lengths(entity_length_df, "Median Length", dataset_mapping)


In [16]:
# Define parameters
datasets_of_interest = ['ANERCorp_CamelLab', 'conll2003']
schemes = ['IOB1', 'IOB2']
splits = ['train', 'test']
dataset_mapping = {
    'ANERCorp_CamelLab': 'ANERCorp',
    'conll2003': 'CoNLL-2003'
    }

# Initialize Analyzer
length_analyzer = EntityLengthAnalyzer(corpora, datasets_of_interest, schemes, splits)

# Compute entity length distributions
entity_length_df = length_analyzer.process_datasets()

# Visualize entity length distributions
length_analyzer.visualize_entity_lengths(entity_length_df, "Interquartile Range (IQR)", dataset_mapping)


In [17]:
# Define parameters
datasets_of_interest = ['ANERCorp_CamelLab', 'conll2003']
schemes = ['IOB1', 'IOB2']
splits = ['train', 'test']
dataset_mapping = {
    'ANERCorp_CamelLab': 'ANERCorp',
    'conll2003': 'CoNLL-2003'
    }

# Initialize Analyzer
length_analyzer = EntityLengthAnalyzer(corpora, datasets_of_interest, schemes, splits)

# Compute entity length distributions
entity_length_df = length_analyzer.process_datasets()

# Visualize entity length distributions
length_analyzer.visualize_entity_lengths(entity_length_df, "Max Length", dataset_mapping)


In [18]:
# Define parameters
datasets_of_interest = ['ANERCorp_CamelLab', 'conll2003']
schemes = ['IOB1', 'IOB2']
splits = ['train', 'test']
dataset_mapping = {
    'ANERCorp_CamelLab': 'ANERCorp',
    'conll2003': 'CoNLL-2003'
    }

# Initialize Analyzer
length_analyzer = EntityLengthAnalyzer(corpora, datasets_of_interest, schemes, splits)

# Compute entity length distributions
entity_length_df = length_analyzer.process_datasets()

# Visualize entity length distributions
length_analyzer.visualize_entity_lengths(entity_length_df, "Mean Length", dataset_mapping)


In [19]:
entity_length_df

,Mean Length,Median Length,Standard Deviation,Interquartile Range (IQR),Mode Length,Min Length,Max Length,Dataset,Scheme,Split,Entity Type
0,1.1381,1.0,0.3815,0.0,1,1,4,ANERCorp,IOB1,Train,LOC
1,1.6967,2.0,0.8000,1.0,1,1,6,ANERCorp,IOB1,Train,ORG
2,1.8011,2.0,0.9126,1.0,2,1,9,ANERCorp,IOB1,Train,PER
3,1.4033,1.0,0.7678,1.0,1,1,6,ANERCorp,IOB1,Train,MISC
4,1.1382,1.0,0.3816,0.0,1,1,4,ANERCorp,IOB2,Train,LOC
5,1.7005,2.0,0.8008,1.0,1,1,6,ANERCorp,IOB2,Train,ORG
6,1.8049,2.0,0.9132,1.0,2,1,9,ANERCorp,IOB2,Train,PER
7,1.4054,1.0,0.7667,1.0,1,1,6,ANERCorp,IOB2,Train,MISC
8,1.5860,1.0,0.9225,1.0,1,1,10,CoNLL-2003,IOB1,Train,ORG
9,1.3360,1.0,0.6773,0.0,1,1,7,CoNLL-2003,IOB1,Train,MISC


In [67]:

y_true = [sen['tags'] for sen in corpora['conll2003']['splits']['train']]
true_entities = get_entities(y_true)
        
scheme = auto_detect(y_true, False)
strict_true_entities = Entities(y_true, scheme, False)
entity_stats = calculate_entity_lengths(true_entities)
strict_entity_stats = calculate_entity_lengths([entity.to_tuple()[1:] for sen in strict_true_entities.entities for entity in sen], True)
display(pd.DataFrame(entity_stats))
display(pd.DataFrame(strict_entity_stats))


,ORG,MISC,PER,LOC
Average Length,1.59,1.34,1.69,1.16
Median Length,1.00,1.00,2.00,1.00
Min Length,1.00,1.00,1.00,1.00
Max Length,10.00,7.00,6.00,10.00
STD,0.92,0.68,0.55,0.44


,ORG,MISC,PER,LOC
Average Length,1.59,1.34,1.69,1.16
Median Length,1.00,1.00,2.00,1.00
Min Length,1.00,1.00,1.00,1.00
Max Length,10.00,7.00,6.00,10.00
STD,0.92,0.68,0.55,0.44


In [82]:
# for every new sentence an increase in the count, for example the second sentence PER entity should be 9, 10 but the count increased and then became 10, 11 then the count increased by one 11, to 12 then 13, etc. 
y_true

[['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O'],
 ['B-PER', 'I-PER'],
 ['B-LOC', 'O'],
 ['O',
  'B-ORG',
  'I-ORG',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-MISC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-MISC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-LOC',
  'O',
  'O',
  'O',
  'O',
  'B-ORG',
  'I-ORG',
  'O',
  'O',
  'O',
  'B-PER',
  'I-PER',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-ORG',
  'O',
  'O',
  'O',
  'B-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-ORG',
  'I-ORG',
 

In [81]:
true_entities

[('ORG', 0, 0),
 ('MISC', 2, 2),
 ('MISC', 6, 6),
 ('PER', 10, 11),
 ('LOC', 13, 13),
 ('ORG', 17, 18),
 ('MISC', 25, 25),
 ('MISC', 31, 31),
 ('LOC', 47, 47),
 ('ORG', 52, 53),
 ('PER', 57, 58),
 ('LOC', 70, 70),
 ('ORG', 99, 99),
 ('PER', 103, 106),
 ('ORG', 135, 136),
 ('ORG', 146, 146),
 ('PER', 149, 150),
 ('PER', 180, 180),
 ('MISC', 182, 182),
 ('LOC', 187, 187),
 ('LOC', 189, 189),
 ('MISC', 197, 199),
 ('MISC', 201, 201),
 ('PER', 210, 210),
 ('ORG', 218, 218),
 ('MISC', 247, 247),
 ('PER', 250, 252),
 ('PER', 256, 256),
 ('ORG', 259, 259),
 ('LOC', 278, 278),
 ('LOC', 280, 280),
 ('PER', 282, 282),
 ('ORG', 288, 288),
 ('MISC', 328, 328),
 ('MISC', 345, 345),
 ('MISC', 365, 365),
 ('MISC', 372, 372),
 ('LOC', 378, 378),
 ('LOC', 398, 398),
 ('ORG', 403, 407),
 ('ORG', 409, 409),
 ('PER', 412, 414),
 ('ORG', 417, 418),
 ('LOC', 421, 421),
 ('MISC', 437, 437),
 ('LOC', 456, 456),
 ('LOC', 461, 461),
 ('MISC', 478, 478),
 ('PER', 490, 490),
 ('LOC', 500, 500),
 ('LOC', 512, 512)

In [68]:
entity_lengths = defaultdict(list)
strict = False
# Compute entity lengths per entity type
for entity_type, start, end in true_entities:
    length = end - start if strict else end - start + 1  # Adjust for inclusive indexing
    entity_lengths[entity_type.replace('PERS', 'PER')].append(length)

In [85]:
Counter(entity_lengths['ORG'])

Counter({1: 3836, 2: 1705, 3: 507, 4: 173, 5: 59, 6: 21, 7: 17, 8: 2, 10: 1})